In [ ]:
from selenium import webdriver
import time
import re
from selenium.common.exceptions import NoSuchElementException
import requests
import pandas as pd
import os
from pathlib import Path

# Create Folder Structure

In [ ]:
try:
    os.makedirs(".\\rGSW\\")
    os.makedirs(".\\rHOU\\")
    os.makedirs(".\\rMEM\\")
    os.makedirs(".\\rPHX\\")
    os.makedirs(".\\GSW\\")
    os.makedirs(".\\HOU\\")
    os.makedirs(".\\MEM\\")
    os.makedirs(".\\PHX\\")
except FileExistsError:
    # directory already exists
    pass

# Get schedule

In [ ]:
def crawlSchedule(url, rawFinds, team):
    siteRequest = requests.get(url)
    #print(siteRequest.text)


    for item in re.findall(r"(month=\d{1,5}&amp;day=\d{1,5}&amp;year=\d{1,5}).+" + team, siteRequest.text, re.MULTILINE):
        rawFinds.append(item)
    return rawFinds


In [ ]:
def getScheduleForTeam(chosenTeam):
    rawFinds = []
    urls = ['https://www.basketball-reference.com/leagues/NBA_2018_games-october.html', 
           'https://www.basketball-reference.com/leagues/NBA_2018_games-november.html', 
           'https://www.basketball-reference.com/leagues/NBA_2018_games-december.html', 
           'https://www.basketball-reference.com/leagues/NBA_2018_games-january.html', 
           'https://www.basketball-reference.com/leagues/NBA_2018_games-february.html', 
           'https://www.basketball-reference.com/leagues/NBA_2018_games-march.html', 
           'https://www.basketball-reference.com/leagues/NBA_2018_games-april.html', 
           'https://www.basketball-reference.com/leagues/NBA_2018_games-may.html', 
           'https://www.basketball-reference.com/leagues/NBA_2018_games-june.html']

    for url in urls:
        crawlSchedule(url, rawFinds, chosenTeam)
    
    schedule = []

    for find in rawFinds:
        dateParts = re.findall(r"\d{1,4}", find, re.MULTILINE)
        
        if(int(dateParts[2]) < 2018):
            schedule.append(dateParts[2] + "_" + dateParts[0] + "_" + dateParts[1])
            continue
        
        if(int(dateParts[0]) <= 4):
            if(int(dateParts[0]) == 4):
                if(int(dateParts[1]) > 11):
                    continue
        else:
            continue
        
        schedule.append(dateParts[2] + "_" + dateParts[0] + "_" + dateParts[1])

    return schedule

## Get schedule for Rockets season 17/18

In [ ]:
rocketsSchedule = getScheduleForTeam('Houston Rockets')
print(rocketsSchedule)
print("number of games: " + str(len(rocketsSchedule)))

## Get Off Rtg for schedule

In [ ]:
def getOffRtg(teamNumber, schedule):
    offRtgList = []
    
    for gameDate in schedule:
        print(gameDate)
        year = gameDate.split('_')[0]
        month = gameDate.split('_')[1]
        day = gameDate.split('_')[2]
        
        
        url = "https://www.nba.com/stats/team/" + teamNumber + "/advanced/?Season=2017-18&SeasonType=Regular%20Season&DateFrom=" + month + "%2F" + day + "%2F" + year + "&DateTo="+ month + "%2F" + day + "%2F" + year
        
        driver = webdriver.Chrome("chromedriver.exe") #path of selenium driver
        driver.get(url)
        time.sleep(5)
        
        pageHtml = driver.page_source
        
        
        findings = re.findall(r'<tr data-ng-repeat=".+[\n ]+<td.+[\n \d\-<]+<\/td>[\n \d\-\.]+<td[ a-zA-z="]{0,15}>[\n \d\-\.]+<\/td>[\n \d\-\.]+<td[ a-zA-z="]{0,15}>[\n \d\-\.]+<\/td>[\n \d\-\.]+<td[ a-zA-z="]{0,15}>([\w\.]+)', pageHtml, re.MULTILINE)
        
        
        
        if(len(findings) < 1):
            driver.get(url)
            time.sleep(10)
            finding = re.findall(r'<tr data-ng-repeat=".+[\n ]+<td.+[\n \d\-<]+<\/td>[\n \d\-\.]+<td[ a-zA-z="]{0,15}>[\n \d\-\.]+<\/td>[\n \d\-\.]+<td[ a-zA-z="]{0,15}>[\n \d\-\.]+<\/td>[\n \d\-\.]+<td[ a-zA-z="]{0,15}>([\w\.]+)', pageHtml, re.MULTILINE)[0]
        else:
            finding = findings[0]
        
        pageHtml = driver.page_source
        
        offRtgList.append(finding)
        
    return offRtgList

In [ ]:
offRtgListRockets = getOffRtg('1610612745', rocketsSchedule)

In [ ]:
def printOffRtgToCSV(schedule, offRtgList, teamAcronym):
    output = "Date;OffRtg\n"
    index = 0
    
    for gameDate in schedule:
        output = output + gameDate + ";" + offRtgList[index]
        if(index < 81):
            output = output + "\n"
        
        index = index + 1
        
    with open(".\\" + teamAcronym + "\\offRtg_" + teamAcronym + ".csv", "w") as text_file:
        text_file.write(output)
        
        
    print("csv printed")
    
    

In [ ]:
printOffRtgToCSV(rocketsSchedule, offRtgListRockets, "HOU")

# Get list of Players for date

In [ ]:
def getPlayerList(date, startPlayerNumber, teamAcronym):
    year = date.split('_')[0]
    month = date.split('_')[1]
    day = date.split('_')[2]

    url = 'https://www.nba.com/stats/player/' + startPlayerNumber + '/passes-dash/?Season=2017-18&SeasonType=Regular%20Season&DateFrom=' + month +'%2F' + day +'%2F' + year +'&DateTo=' + month +'%2F' + day +'%2F' + year

    print(url)
    
    driver = webdriver.Chrome("chromedriver.exe") #path of selenium driver
    driver.get(url)
    time.sleep(2)

    pageHtml = driver.page_source # notwendig, da javascript elemente nachladet
    #print(pageHtml)
    
    return list(set(re.findall(r'<a href="\/stats\/player\/(\w+)\/passes-dash\/">[A-z, \n\.\-\']+<\/a><\/span><!---->[<>\/="A-z, \n\d-]+' + teamAcronym, pageHtml, re.MULTILINE)))


In [ ]:
def getPlayerList2(date, teamNumber):
    year = date.split('_')[0]
    month = date.split('_')[1]
    day = date.split('_')[2]

    url = 'https://www.nba.com/stats/team/' + teamNumber + '/players-traditional/?Season=2017-18&SeasonType=Regular%20Season&DateFrom=' + month +'%2F' + day +'%2F' + year +'&DateTo=' + month +'%2F' + day +'%2F' + year
    
    print(url)
    
    driver = webdriver.Chrome("C:\\Users\\Toby\\Documents\\studium\\MA_2\\SNA\\chromedriver.exe")
    driver.get(url)
    time.sleep(4)

    pageHtml = driver.page_source # notwendig, da javascript elemente nachladet
    
    return list(set(re.findall(r'stats\/player\/(\w{3,10})\/', pageHtml, re.MULTILINE)))

In [ ]:
getPlayerList(rocketsSchedule[0], "201935", "HOU")

## Spielerdaten abziehen / Iteration durch alle Spieler im Spiel

In [ ]:
def getPassingDataForDate(date, playerNumbers, teamAcronym):    
    outputCSV = "PASS FROM;PASS TO;TEAM;FREQ M;PASS;AST;FGM;FGA;FG%;2FGM;2FGA;2FG%;3FGM;3FGA;3FG%\n"
    year = date.split('_')[0]
    month = date.split('_')[1]
    day = date.split('_')[2]
    
    attendingPlayerList = []
    
    for number in playerNumbers:
        #attendingPlayerList = []
        driver = webdriver.Chrome("chromedriver.exe") #path of selenium driver
        driver.get('https://www.nba.com/stats/player/' + number + '/passes-dash/?Season=2017-18&SeasonType=Regular%20Season&DateFrom=' + month +'%2F' + day +'%2F' + year +'&DateTo=' + month +'%2F' + day +'%2F' + year)
        print('https://www.nba.com/stats/player/' + number + '/passes-dash/?Season=2017-18&SeasonType=Regular%20Season&DateFrom=' + month +'%2F' + day +'%2F' + year +'&DateTo=' + month +'%2F' + day +'%2F' + year)
        time.sleep(4)
        
        finds = driver.find_elements_by_xpath('/html/body/main/div/div/div/div[4]/div/div/div/div/nba-stat-table[1]/div[2]/div/table/tbody')
        if(len(finds) <= 0):
            finds = driver.find_elements_by_xpath('/html/body/main/div/div/div/div[4]/div/div/div/div/nba-stat-table[1]/div[2]/div[1]/table/tbody')
            if(len(finds) <= 0):
                finds = driver.find_elements_by_xpath('/html/body/main/div/div/div/div[4]/div/div/div/div/nba-stat-table[1]/div[2]/div[2]/table/tbody')
                if(len(finds) <= 0):
                    finds = driver.find_elements_by_xpath('/html/body/main/div/div/div/div[4]/div/div/div/div/nba-stat-table[1]/div[2]/div[2]/table/tbody')
                    if(len(finds) <= 0): # keine daten in der tabelle des spielers vorhanden
                        continue
                        
                    rowsOfPlayer = driver.find_element_by_xpath('/html/body/main/div/div/div/div[4]/div/div/div/div/nba-stat-table[1]/div[2]/div[1]/table/tbody')
                else:
                    rowsOfPlayer = driver.find_element_by_xpath('/html/body/main/div/div/div/div[4]/div/div/div/div/nba-stat-table[1]/div[2]/div[2]/table/tbody')
            
            else:
                rowsOfPlayer = driver.find_element_by_xpath('/html/body/main/div/div/div/div[4]/div/div/div/div/nba-stat-table[1]/div[2]/div[1]/table/tbody')
        else:
            rowsOfPlayer = driver.find_element_by_xpath('/html/body/main/div/div/div/div[4]/div/div/div/div/nba-stat-table[1]/div[2]/div/table/tbody')

        tbodyRip =  rowsOfPlayer.get_attribute('innerHTML')

        pageHTML = driver.page_source

        print(re.findall('<div class="stats-teamplayer-summary-text large">\\n{0,10}\\\\?n? *([a-zA-Z]{1,12})', pageHTML))
        firstName = re.findall('<div class="stats-teamplayer-summary-text large">\\n{0,10}\\\\?n? *([a-zA-Z\.\- ]{1,12})', pageHTML)[0]
        lastName = re.findall('<div class="stats-teamplayer-summary-text large">\\n{0,10}\\\\?n? *([a-zA-Z\.\- ]{1,12})', pageHTML)[1]
        outputRip = re.findall('<td>(\d{1,4}\.{0,1}\d)|<td>(-)<\/td>|>(\w{0,7}[\. \w\-\']{1,10}\w{0,10}, \w{0,7}[\. \w\-\']{1,10}\w{0,10}\w{0,10})<\/a>|>([A-Z]{2,4})<\/a>|dash\/">([A-z ,\."]{1,20})<\/a>', tbodyRip)
        #print(outputRip)
        counter = 0

        driver.close()
        
        attendingPlayerList.append(lastName + ", " + firstName)

        
        for rip in outputRip:
            for finding in rip:
                if finding:
                    #ausnahme, da namen inkonsistent auf der seite auftauchen
                    if finding == "Nene":
                        finding = "Hilario, Nene"
                    if finding == "Zhou Qi":
                        #print("here")
                        finding = "Qi, Zhou"
                    
                    if counter == 0:
                        if firstName == "Nene":
                            lastname = "Hilario"
                        
                        outputCSV = outputCSV + lastName + ", " + firstName + ";"
                    else:
                        outputCSV = outputCSV + ";"
                    
                    if(len(re.findall(r'[A-Za-z, ]{4,30}', finding)) > 0):
                        attendingPlayerList.append(finding)

                    outputCSV = outputCSV + finding

                    counter = counter + 1

                    if counter == 14:
                        outputCSV = outputCSV + "\n"
                        counter = 0

        outputCSV = outputCSV + "\n"
        counter = 0
        print(outputCSV)
        print("---------------------")
        
    #print data to csv
    outputCSV = outputCSV.replace('\n\n', '\n')
    with open(".\\" + teamAcronym + "\\passing_" + date + "_" + teamAcronym + ".csv", "w") as text_file:
        text_file.write(outputCSV)
        
    #print nodes to csv
    dfPlayerNodes = pd.DataFrame(list(set(attendingPlayerList)), columns =['name'])
    dfPlayerNodes.to_csv(".\\" + teamAcronym + "\\nodes_" + date + "_" + teamAcronym + ".csv", sep=';')
    

In [ ]:
playerNumbers = []

teamAcronym = 'HOU'

with open(filename) as file:
    lines = file.readlines()
    for line in lines:
        searchStartPlayer.append(line.replace("\n", ""))
        
searchStartPlayer = list(set(searchStartPlayer))

for schedule in rocketsSchedule:
    playerNumbers = []
    
    
    playerNumbers.extend(getPlayerList2(schedule, "1610612745"))
    
    
    
    print(list(set(playerNumbers)))
    #break
    
    getPassingDataForDate(schedule, list(set(playerNumbers)), teamAcronym)
    
    #break
    skipper = True
    

In [ ]:
print("done")

# Golden State

In [ ]:
teamAcronym = 'GSW'

## Create Folder

In [ ]:
Path(".\\" + teamAcronym).mkdir(parents=True, exist_ok=True)

## Get Schedule

In [ ]:
#https://www.basketball-reference.com/leagues/NBA_2018_games-october.html
goldenStateSchedule = getScheduleForTeam('Golden State Warriors')
print(goldenStateSchedule)
print("number of games: " + str(len(goldenStateSchedule)))

## Get Off Rtg for schedule

In [ ]:
#https://www.nba.com/stats/team/1610612745/advanced/?Season=2017-18&SeasonType=Regular%20Season
offRtgListGoldenState = getOffRtg('1610612744', goldenStateSchedule)

In [ ]:
offRtgListGoldenState

In [ ]:
printOffRtgToCSV(goldenStateSchedule, offRtgListGoldenState, teamAcronym)

## Spielerdaten abziehen / Iteration durch alle Spieler im Spiel

In [ ]:
skipper = True

playerNumbers = []


for schedule in goldenStateSchedule:
    playerNumbers = []
    
    playerNumbers.extend(getPlayerList2(schedule, "1610612744"))
    
    #break;
    
    print(list(set(playerNumbers)))
    #break
    
    getPassingDataForDate(schedule, list(set(playerNumbers)), teamAcronym)
    
    #break
    skipper = True
    

In [ ]:
print("done")

# Suns

In [ ]:
teamAcronym = 'PHX'

## Create Folder

In [ ]:
Path(".\\" + teamAcronym).mkdir(parents=True, exist_ok=True)

## Get Schedule

In [ ]:
#https://www.basketball-reference.com/leagues/NBA_2018_games-october.html
sunsSchedule = getScheduleForTeam('Phoenix Suns')
print(sunsSchedule)
print("number of games: " + str(len(sunsSchedule)))

## Get Off Rtg for schedule

In [ ]:
#https://www.nba.com/stats/team/1610612745/advanced/?Season=2017-18&SeasonType=Regular%20Season
offRtgListSuns = getOffRtg('1610612756', sunsSchedule)

In [ ]:
offRtgListSuns

In [ ]:
printOffRtgToCSV(sunsSchedule, offRtgListSuns, teamAcronym)

## Spielerdaten abziehen / Iteration durch alle Spieler im Spiel

In [ ]:
skipper = True

playerNumbers = []


for schedule in sunsSchedule:
    playerNumbers = []
    
    playerNumbers.extend(getPlayerList2(schedule, "1610612756"))
    
    #break;
    
    print(list(set(playerNumbers)))
    #break
    
    getPassingDataForDate(schedule, list(set(playerNumbers)), teamAcronym)
    
    #break
    #skipper = True
    

# Kings

In [ ]:
teamAcronym = 'SAC'

## Create Folder

In [ ]:
Path(".\\" + teamAcronym).mkdir(parents=True, exist_ok=True)

## Get Schedule

In [ ]:
#https://www.basketball-reference.com/leagues/NBA_2018_games-october.html
kingsSchedule = getScheduleForTeam('Sacramento Kings')
print(kingsSchedule)
print("number of games: " + str(len(kingsSchedule)))

## Get Off Rtg for schedule

In [ ]:
#https://www.nba.com/stats/team/1610612745/advanced/?Season=2017-18&SeasonType=Regular%20Season
offRtgListKings = getOffRtg('1610612758', kingsSchedule)

In [ ]:
offRtgListKings

In [ ]:
printOffRtgToCSV(kingsSchedule, offRtgListKings, teamAcronym)

## Spielerdaten abziehen / Iteration durch alle Spieler im Spiel

In [ ]:
skipper = True

playerNumbers = []


for schedule in kingsSchedule:
    playerNumbers = []
    
    playerNumbers.extend(getPlayerList2(schedule, "1610612758"))
    
    
    #break;
    
    
    print(list(set(playerNumbers)))
    #break
    
    getPassingDataForDate(schedule, list(set(playerNumbers)), teamAcronym)
    
    #break
    #skipper = True
    

In [ ]:
print("done")

# Grizzlies

In [ ]:
teamAcronym = 'MEM'

## Create Folder

In [ ]:
Path(".\\" + teamAcronym).mkdir(parents=True, exist_ok=True)

## Get Schedule

In [ ]:
#https://www.basketball-reference.com/leagues/NBA_2018_games-october.html
grizzliesSchedule = getScheduleForTeam('Memphis Grizzlies')
print(grizzliesSchedule)
print("number of games: " + str(len(grizzliesSchedule)))

## Get Off Rtg for schedule

In [ ]:
#https://www.nba.com/stats/team/1610612745/advanced/?Season=2017-18&SeasonType=Regular%20Season
offRtgListGrizzlies = getOffRtg('1610612763', grizzliesSchedule)

In [ ]:
offRtgListGrizzlies

In [ ]:
printOffRtgToCSV(grizzliesSchedule, offRtgListGrizzlies, teamAcronym)

## Spielerdaten abziehen / Iteration durch alle Spieler im Spiel

In [ ]:
skipper = True

playerNumbers = []


for schedule in grizzliesSchedule:
    playerNumbers = []
    
    if schedule == '2018_3_19':
        skipper = False
        print("cur: " + schedule)
       
    if skipper == True:
        print("continue")
        continue
        
    playerNumbers.extend(getPlayerList2(schedule, "1610612763"))
    
    #break;
    
    print(list(set(playerNumbers)))
    #break
    
    #break
    getPassingDataForDate(schedule, list(set(playerNumbers)), teamAcronym)
    
    #break
    #skipper = True
    

In [ ]:
print("done")